In [113]:
import yfinance as yf
import pandas as pd
import numpy as np

In [114]:
# 步驟1：資料蒐集
# 下載台積電 (2330.TW) 和富邦金 (2881.TW) 從2020年1月1日到2023年1月1日的歷史股價資料
data = yf.download(['2330.TW', '2881.TW'], start='2020-01-01', end='2023-01-01')

# 計算日收益率
returns = data['Adj Close'].pct_change()

# 計算平均日收益率和日收益率標準差（波動率）
mean_returns = returns.mean()
print("平均日收益:", mean_returns)

std_dev_returns = returns.std()
print("日收益率標準差:", std_dev_returns)

# 計算相關係數
correlation = returns.corr()
print("相關係數矩陣:", correlation)

[*********************100%***********************]  2 of 2 completed

平均日收益: Ticker
2330.TW    0.000638
2881.TW    0.000789
dtype: float64
日收益率標準差: Ticker
2330.TW    0.018222
2881.TW    0.016112
dtype: float64
相關係數矩陣: Ticker    2330.TW   2881.TW
Ticker                     
2330.TW  1.000000  0.462788
2881.TW  0.462788  1.000000


In [115]:
# 染色體表示和初始種群生成
def initialize_population(num_portfolios, num_assets):
    population = np.random.dirichlet(np.ones(num_assets), size=num_portfolios)
    return population

# 適應度函數（假設以夏普比率為適應度評估）
def fitness(population, mean_returns, std_dev_returns, risk_free_rate=0.05):
    portfolio_returns = np.dot(population, mean_returns)
    portfolio_std_dev = np.sqrt(np.dot(population**2, std_dev_returns**2))
    sharpe_ratio = (portfolio_returns - risk_free_rate) / portfolio_std_dev
    return sharpe_ratio

# 選擇
def selection(population, fitness_scores):
    probabilities = fitness_scores / fitness_scores.sum()
    selected_indices = np.random.choice(np.arange(len(population)), size=len(population), p=probabilities)
    selected_population = population[selected_indices]
    return selected_population

# 交叉
def crossover(population, crossover_rate=0.6):
    crossed_population = []
    for i in range(0, len(population), 2):
        if i + 1 < len(population) and np.random.rand() < crossover_rate:
            cross_point = np.random.randint(1, len(population[i]))
            new_individual1 = np.concatenate([population[i][:cross_point], population[i+1][cross_point:]])
            new_individual2 = np.concatenate([population[i+1][:cross_point], population[i][cross_point:]])
            crossed_population.extend([new_individual1, new_individual2])
        else:
            crossed_population.extend([population[i], population[i+1]])
    return np.array(crossed_population)

# 變異
def mutation(population, mutation_rate=0.05):
    mutated_population = []
    for individual in population:
        if np.random.rand() < mutation_rate:
            mutation_point = np.random.randint(len(individual))
            individual[mutation_point] = np.random.rand()
            individual /= individual.sum()  # 重新正規化以保持總和為1
        mutated_population.append(individual)
    return np.array(mutated_population)

# 執行基因演算法
num_assets = 2
num_portfolios = 100
num_generations = 100

# 初始化
population = initialize_population(num_portfolios, num_assets)
for _ in range(num_generations):
    # 計算適應度
    fitness_scores = fitness(population, mean_returns, std_dev_returns)
    # 選擇
    population = selection(population, fitness_scores)
    # 交叉
    population = crossover(population)
    # 變異
    population = mutation(population)

# 計算最終適應度
final_fitness_scores = fitness(population, mean_returns, std_dev_returns)
best_portfolio_index = np.argmax(final_fitness_scores)
best_portfolio = population[best_portfolio_index]
print("最優投資組合:", {(data.columns[i])[1]: best_portfolio[i] for i in range(num_assets)})
print("最優適應度:", final_fitness_scores[best_portfolio_index])


最優投資組合: {'2330.TW': 0.9718783311661204, '2881.TW': 0.028121668833879575}
最優適應度: -2.7860581681261865
